# Fine Tune Stable Diffusion

Fine tuning Stable Diffusion on Pokemon, 
for more details see the [Lambda Labs examples repo](https://github.com/LambdaLabsML/examples). 

We recommend using a multi-GPU machine, for example an instance from [Lambda GPU Cloud](https://lambdalabs.com/service/gpu-cloud). If running on Colab this notebook is likely to need a GPU with >16GB of VRAM and a runtime with high RAM, which will almost certainly need Colab Pro or Pro+. (If you get errors suchs as `Killed` or `CUDA out of memory` then one of these is not sufficient)

## check gpu 

In [8]:
!ls

'Ideal Images Datasets-20221019T022432Z-001.zip'
 ideal_images
'nft-finetune (1).ipynb'
'pokemon_finetune (1) (1).ipynb'


In [9]:
!git clone https://github.com/Jumabek/stable-diffusion.git

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 1631, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 1631 (delta 3), reused 15 (delta 2), pack-reused 1603
Receiving objects: 100% (1631/1631), 73.93 MiB | 19.00 MiB/s, done.
Resolving deltas: 100% (982/982), done.


In [10]:
%cd /home/ubuntu/stable-diffusion
!pip install --upgrade pip
!pip install -r requirements.txt

!pip install --upgrade keras # on lambda stack we need to upgrade keras
!pip uninstall -y torchtext # on colab we need to remove torchtext

/home/ubuntu/stable-diffusion
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 24))
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone --filter=blob:none --quiet https://github.com/CompVis/taming-transformers.git /home/ubuntu/stable-diffusion/src/taming-transformers
  Resolved https://github.com/CompVis/taming-transformers.git to commit 24268930bf1dce879235a7fddd0b2355b84d7ea6
  Preparing metadata (setup.py) ... done
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip (from -r requirements.txt (line 25))
  Cloning https://github.com/openai/C

In [7]:
!ls

'Ideal Images Datasets-20221019T022432Z-001.zip'
 ideal_images
'nft-finetune (1).ipynb'
'pokemon_finetune (1) (1).ipynb'


To get the weights you need to you'll need to [go to the model card](https://huggingface.co/CompVis/stable-diffusion-v1-4-original), read the license and tick the checkbox if you agree.

In [11]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

Defaulting to user installation because normal site-packages is not writeable


In [12]:
from huggingface_hub import hf_hub_download
ckpt_path = hf_hub_download(repo_id="CompVis/stable-diffusion-v-1-4-original", filename="sd-v1-4-full-ema.ckpt", use_auth_token=True)

Downloading:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

Set your parameters below depending on your GPU setup, the settings below were used for training on a 2xA6000 machine, (the A6000 has 48GB of VRAM). On this set up good results are achieved in around 6 hours.

You can make up for using smaller batches or fewer gpus by accumulating batches:

`total batch size = batach size * n gpus * accumulate batches`

In [13]:
# 2xA6000:
BATCH_SIZE = 4
N_GPUS = 8
ACCUMULATE_BATCHES = 1

gpu_list = ",".join((str(x) for x in range(N_GPUS))) + ","
print(f"Using GPUs: {gpu_list}")
gpu_list

Using GPUs: 0,1,2,3,4,5,6,7,


'0,1,2,3,4,5,6,7,'

## Fine tuning 

In [27]:
# Run training
%cd /home/ubuntu/stable-diffusion
!(python main.py \
    -t \
    --base configs/stable-diffusion/nft.yaml \
    --gpus '2,3,' \
    --scale_lr False \
    --num_nodes 1 \
    --check_val_every_n_epoch 10 \
    --finetune_from "$ckpt_path" \
    data.params.batch_size=2 \
    lightning.trainer.accumulate_grad_batches=1 \
    data.params.validation.params.n_gpus=1 \
)

/home/ubuntu/stable-diffusion
Global seed set to 23
Running on GPUs 2,3,
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.mlp.fc1.weight', 'vision_model.encoder.layers.10.layer_norm2.weight', 'vision_model.encoder.layers.11.layer_norm2.weight', 'vision_model.encoder.layers.12.self_attn.v_proj.bias', 'vision_model.encoder.layers.19.mlp.fc2.bias', 'vision_model.encoder.layers.2.self_attn.q_proj.weight', 'text_projection.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.bias', 'vision_model.encoder.layers

## Inference

In [13]:
# Run the model
!(python scripts/txt2img.py \
    --prompt 'robotic cat with wings' \
    --outdir 'outputs/generated_pokemon' \
    --H 512 --W 512 \
    --n_samples 4 \
    --config 'configs/stable-diffusion/pokemon.yaml' \
    --ckpt 'path/to/your/checkpoint')

Global seed set to 42
Loading model from path/to/your/checkpoint
Traceback (most recent call last):
  File "scripts/txt2img.py", line 285, in <module>
    main()
  File "scripts/txt2img.py", line 194, in main
    model = load_model_from_config(config, f"{opt.ckpt}")
  File "scripts/txt2img.py", line 27, in load_model_from_config
    pl_sd = torch.load(ckpt, map_location="cpu")
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/serialization.py", line 699, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/checkpoint'
